# Lab 2: MLR with Dummy Variables

## Instructions

Copy and paste the following code into a new R script file in RStudio. Save the file as `lab2_binary_variables.R` (we call it a script). Then, run the code step by step, answering the questions.

In [5]:
# load packages and dataset
pkgs <- c("tidyverse", "moments", "data.table", "ggsci", "stargazer")
missing <- setdiff(pkgs, rownames(installed.packages()))
if (length(missing) > 0) install.packages(missing)
invisible(lapply(pkgs, function(pkg) suppressPackageStartupMessages(library(pkg, character.only = TRUE))))

## Load data into working directory

We continue to use the same dataset California school dataset (CASchools). Refer to [Data Visualization: Dataset Overview](02_Data_Visualization.qmd#dataset-overview) for detailed description of the dataset.

Previously, we have loaded the data directly from an online data repository. Now, we will load the data from a local CSV file. Downloaded the `CASchools_test_score.csv` file from Canvas and placed it in the same folder as where you put `lab2_binary_variables.R`. This is to make sure that the script can find the data file.

In [6]:
# load dataset
f_name <- "CASchools_test_score.csv"
cas <- read_csv(f_name)
# data preview, first and last 5 rows
cas %>% as.data.table() %>% print(topn=5)

     district                          school      county grades students
        <num>                          <char>      <char> <char>    <num>
  1:    75119              Sunol Glen Unified     Alameda  KK-08      195
  2:    61499            Manzanita Elementary       Butte  KK-08      240
  3:    61549     Thermalito Union Elementary       Butte  KK-08     1550
  4:    61457 Golden Feather Union Elementary       Butte  KK-08      243
  5:    61523        Palermo Union Elementary       Butte  KK-08     1335
 ---                                                                     
416:    68957          Las Lomitas Elementary   San Mateo  KK-08      984
417:    69518            Los Altos Elementary Santa Clara  KK-08     3724
418:    72611          Somis Union Elementary     Ventura  KK-08      441
419:    72744               Plumas Elementary        Yuba  KK-08      101
420:    72751            Wheatland Elementary        Yuba  KK-08     1778
     teachers calworks   lunch compute

Prepare dataset for analysis. 

As done previously, we create the following variables:

- `TestScore`: average test score of students in a school (average of math and reading scores)
- `str`: student-teacher ratio

In [9]:
cas <- cas %>%
    mutate(
        TestScore = (read + math) / 2,
        STR = students / teachers
    )

## Regression When $X$ is a Binary Variable

Instead of using a continuous regressor $X$ , we might be interested in running the regression

$$
Y_i = \beta_0 + \beta_1 D_i + u_i, 
$$ {#eq-binary-regression}

where $D_i$ is a binary variable, a so-called dummary variable. For example, we may defined $D_i$ as follows:

$$
D_i = 
\begin{cases}
    1 & \text{if $STR$ in $i^{th}$ school district < 20} \\
    0 & \text{if $STR$ in $i^{th}$ school district $\geq$ 20}.
\end{cases}
$$ {#eq-dummy-variable}

The regression model now is 

$$
TestScore_i = \beta_0 + \beta_1 D_i + u_i.
$$

### Data Visualization

In [16]:
# Create the dummy variable as defined above
# D=1 (low STR); D=0 (high STR)
cas$D <- cas$STR < 20

In [21]:
# Compute group means
means <- aggregate(TestScore ~ D, data = cas, mean)
means

D,TestScore
<lgl>,<dbl>
FALSE,650.0768
TRUE,657.2462


In [ ]:
#| echo: false
p <- ggplot(cas, aes(x = factor(D), y = TestScore)) +
    # scatter points, spreads the points horizontally so they don't overlap
    geom_jitter(width = 0.1, height = 0, size = 0.5, color = "steelblue") +
    geom_point(
        data = means, aes(x = factor(D), y = TestScore),
        color = "red", size = 2
    ) + # group means
    labs(
        x = expression(D[i]),
        y = "Test Score",
    ) +
    theme_minimal(base_size = 14)
f_name <- "images/dummy_plot.png"
ggsave(f_name, p, width = 6, height = 4, dpi = 300, units = "in")

In [ ]:
#| eval: false
ggplot(cas, aes(x = factor(D), y = TestScore)) +
    # scatter points, spreads the points horizontally so they don't overlap
    geom_jitter(width = 0.1, height = 0, size = 0.5, color = "steelblue") +
    geom_point(
        data = means, aes(x = factor(D), y = TestScore),
        color = "red", size = 2
    ) + # group means
    labs(
        x = expression(D[i]),
        y = "Test Score",
    ) +
    theme_minimal(base_size = 14)

In [47]:
#| echo: false
# Display the image with controlled width and centered
library(IRdisplay)
display_html(paste0('<div style="text-align: center;"><img src="', f_name, '" style="width: 70%; height: auto;"></div>'))

With $D$ as the regressor, it is not useful to think of $\beta_1$ as a slope parameter since since $D_i \in \{0,1\},$ i.e., we only observe two discrete values instead of a continuum of regressor values. There is no continuous line depicting the conditional expectation function $E(TestScore_i | D_i)$ since this function is solely defined for $x$-positions $D_i = 0$ and $D_i = 1$.

Therefore, <span class="env-green">the interpretation</span> of the coefficients in this regression model is as follows:

- $E(Y_i | D_i = 0) = \beta_0$, so $\beta_0$ is the expected test score in districts where $D_i = 0$ and $STR$ is larger or equal to 20.
- $E(Y_i | D_i = 1) = \beta_0 + \beta_1$, or $\beta_1 = E(Y_i | D_i = 1) - E(Y_i | D_i = 0).$ Thus, $\beta_1$ is the difference in group-specific expectations, i.e., the difference in expected test score between districts with $STR<20$ and those with $STR \geq 20.$

We now @eq-binary-regression